In [1]:
from carbon.helpers.stdimports import *
from carbon.helpers.soltest import SolTestBase
from carbon.helpers.floatint import *
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SolTestBase))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonFloatInt32))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonOrderUI))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(P))
from math import log2, floor, ceil, sqrt
print_version(require="2.3.3")

[stdimports] imported np, pd, plt, os, sqrt, exp, log
SolTestBase v1.0 (11/Feb/2023)
CarbonFloatInt32 v1.0 (10/Feb/2023)
CarbonOrderUI v1.7.1 (12/Feb/2023)
CarbonPair v2.2 (11/Feb/2022)
Carbon v2.3.3-BETA1 (12/Feb/2022)


# Solidity Curve Testing (NBTest058)

---

**NOTE: THIS NOTEBOOK DOES NOT CURRENTLY CONTAIN ANY AUTOMATED TESTS**

---


we consider by source and by target **from the point of view of the TRADER**, therefore _by source_ fixes _dx_ and calculates _dy_ and vice versa.

**dx from dy ("by target")**

$$
\Delta x \,(\Delta y) = \frac{\Delta y z^2}{(Ay+Bz)(Ay+Bz-A\Delta y)} =_{A=0} \frac{\Delta y} {B^2}
$$



**dy from dx ("by source")**

$$
\Delta y \,(\Delta x) = \frac{\Delta x(Ay+Bz)^2}{A\Delta x(Ay+Bz)+z^2} =_{A=0} \frac{\Delta x(Bz)^2}{z^2} = \Delta x B^2
$$

[doc][doc]


[doc]:https://docs.google.com/document/d/1x4ZbbS3nIRSJ0ojaOrcBhOc-_eOTga6BajDjK45o7u4/edit

### Setting up the common code

In [2]:
mulDivF = lambda x, y, z: int((x * y) // z)
mulDiv  = mulDivF
mulDivC = lambda x, y, z: int((x * y + z - 1) // z)

In [3]:
def dy_from_dx(params, C):
    """
    calculates trade amount dx (received by AMM) from dy (sold by AMM)
    
    :params:   tuple(dx, (y,z,A,B,S))
    :C:        the check function C(num, label); must return int(num)
    :returns:  dy
    """

    dx = params[0]
    y,z,A,B,ONE = params[1]
    temp1 = C(y * A + z * B, "temp1")               
    temp2 = C(temp1 * dx / ONE, "temp2")            
    temp3 = C(temp2 * A + z * z * ONE, "temp3") 
    dy = mulDiv(temp1, temp2, temp3)
    #print(temp1, temp2, temp3, dy)
    return dy

In [4]:
def dx_from_dy(params, C):
    """
    calculates trade amount dy (sold by AMM) from dx (received by AMM) 
    
    :params:   tuple(dy, (y,z,A,B,S))
    :C:        the check function C(num, label); must return int(num)
    :returns:  dx
    """

    dy = params[0]
    y,z,A,B,ONE = params[1]
    temp1 = C(z * ONE, "temp1")                  
    temp2 = C(y * A + z * B, "temp2")         
    temp3 = C(temp2 - dy * A, "temp3")        
    scale = mulDiv(temp2, temp3, 2**255)+1
    temp1s = C(temp1//scale, "temp1s")
    temp2s = C(temp2//scale, "temp2s")
    dx = mulDiv(
        C(dy*temp1s, "dx*temp1s"), 
        temp1, 
        C(temp2s*temp3, "temp2s*temp3")
    )
    #print(temp1, temp2, temp3, temp1s, temp2s, dy)
    return dx

## DummyTest

In [23]:
assert True

## Running comparison simple examples [NOTEST]

In [5]:
class STB(SolTestBase):
    PRINT_LVL_DEFAULT = SolTestBase.LVL_LOG
    #PRINT_LVL_DEFAULT = SolTestBase.LVL_WARN
VERBOSE = True

### SHIB/BTC (Selling BTC)

In [6]:
PAIR = P("SHIB/BTC").sd(18,8)
price = 1e-5 * 1e-5 # SHIB per BTC
capacity = 1000 # 1000 BTC
oui = CarbonOrderUI.from_prices(PAIR, "BTC", price, price/1.05, capacity, capacity)
curve = oui.yzABS(sx=48, verbose=True)
curve

[yzABS] pair=P('SHIB/BTC').sd(18,8), y=BTC(8), x=SHIB(18)
[yzABS] scale = 2**48 = 281474976710656
[yzABS] y=1000 -> y_wei=100000000000 [BTC-wei]
[yzABS] yint=1000 -> z_wei=100000000000 [BTC-wei]
[yzABS] pa_raw=1.0000000000000002e-10 BTC per SHIB -> 0 BTC-wei per SHIB-wei
[yzABS] pb_raw=9.523809523809526e-11 BTC per SHIB -> 0 BTC-wei per SHIB-wei
[yzABS] a=2.4099927051466823e-07 -> 2.4099927051466825e-12 * scale = 678
[yzABS] b=9.759000729485333e-06 -> 9.759000729485333e-11 * scale = 27469
[yzABS] yzABS = r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656)


r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656)

In [7]:
params  = (1*1e8, curve)       # dy = BTC-wei
dx = dx_from_dy( params, STB() )
dx/1e18 # 1e10 SHIB <- 1 BTC

[LOG:OK:TEMP1] 85 bits: ok
[LOG:OK:TEMP2] 52 bits: ok
[LOG:OK:TEMP3] 52 bits: ok
[LOG:OK:TEMP1S] 85 bits: ok
[LOG:OK:TEMP2S] 52 bits: ok
[LOG:OK:DX*TEMP1S] 112 bits: ok
[LOG:OK:TEMP2S*TEMP3] 103 bits: ok


10000594518.506039

In [8]:
dx2 = oui.dxfromdy_f(1)
print(f"dx={dx/1e18:0.2f} vs {dx2:0.2f} {oui.tknx}, diff={abs(dx/1e18/dx2-1)*100:0.4f}%")

dx=10000594518.51 vs 10000241005.08 SHIB, diff=0.0035%


In [9]:
params = (1e10*1e18, curve)      # dx = SHIB-wei
dy = dy_from_dx(params, STB())
dy/1e8 # 1e3 SHIB <- 1 BTC

[LOG:OK:TEMP1] 52 bits: ok
[LOG:OK:TEMP2] 97 bits: ok
[LOG:OK:TEMP3] 122 bits: ok


0.99994055

In [10]:
dy2 = oui.dyfromdx_f(1e10)
print(f"dy={dy/1e8:0.4f} vs {dy2:0.4f} {oui.tkny}, diff={abs(dy/1e8/dy2-1)*100:0.4f}%")

dy=0.9999 vs 1.0000 BTC, diff=0.0035%


### SHIB/BTC (selling SHIB)

In [11]:
PAIR = P("SHIB/BTC").sd(18,8)
price = 1e-5 * 1e-5 # SHIB per BTC # 1e10 USD
capacity = 1000 * 1e10 # 1000 BTC
oui = CarbonOrderUI.from_prices(PAIR, "SHIB", price, price*1.05, capacity, capacity)
curve = oui.yzABS(sx=40, verbose=True)
curve

[yzABS] pair=P('SHIB/BTC').sd(18,8), y=SHIB(18), x=BTC(8)
[yzABS] scale = 2**40 = 1099511627776
[yzABS] y=10000000000000.0 -> y_wei=1e+31 [SHIB-wei]
[yzABS] yint=10000000000000.0 -> z_wei=1e+31 [SHIB-wei]
[yzABS] pa_raw=9999999999.999996 SHIB per BTC -> 99999999999999967232 SHIB-wei per BTC-wei
[yzABS] pb_raw=9523809523.80952 SHIB per BTC -> 95238095238095208448 SHIB-wei per BTC-wei
[yzABS] a=2409.9927051466802 -> 240999270.51466802 * scale = 264981500216411193344
[yzABS] b=97590.0072948533 -> 9759000729.48533 * scale = 10730134777543587266560
[yzABS] yzABS = r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776)


r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776)

In [12]:
params  = (1*1e5*1e5*1e18, curve)       # dy = SHIB-wei (1USD)
dx = dx_from_dy( params, STB() )
dx/1e8 # 1e10 SHIB = 1 BTC

[LOG:OK:TEMP1] 143 bits: ok
[LOG:OK:TEMP2] 177 bits: ok
[LOG:OK:TEMP3] 177 bits: ok
[LOG:OK:TEMP1S] 46 bits: ok
[LOG:OK:TEMP2S] 79 bits: ok
[LOG:OK:DX*TEMP1S] 139 bits: ok
[WARNING:OVERFLOW:TEMP2S*TEMP3] 255 bits: close to max


1.0000241

In [13]:
dx2 = oui.dxfromdy_f(1)
print(f"dx={dx/1e8:0.6f} vs {dx2:0.6f} {oui.tknx}, diff={abs(dx/1e18/dx2-1)*100:0.4f}%")

dx=1.000024 vs 0.000000 BTC, diff=0.0024%


In [14]:
params = (1e8, curve)      # dx = BTC-wei
dy = dy_from_dx( params, STB() )
dy/1e18/1e10 #1e10 SHIB = 1 BTC

[LOG:OK:TEMP1] 177 bits: ok
[LOG:OK:TEMP2] 163 bits: ok
[WARNING:OVERFLOW:TEMP3] 246 bits: close to max


0.9999759006537406

In [15]:
dy2 = oui.dyfromdx_f(1)
print(f"dy={dy/1e18:0.4f} vs {dy2:0.4f} {oui.tkny}, diff={abs(dy/1e18/dy2-1)*100:0.8f}%")

dy=9999759006.5374 vs 9999759006.5374 SHIB, diff=0.00000000%


## Running the whole tests [NOTEST]

In [16]:
class STB(SolTestBase):
    #PRINT_LVL_DEFAULT = SolTestBase.LVL_LOG
    #PRINT_LVL_DEFAULT = SolTestBase.LVL_WARN
    PRINT_LVL_DEFAULT = SolTestBase.LVL_ERR
VERBOSE = False

### Template code

In [17]:
## SETUP Y = TKNQ, X = TKNB

# parameters
decb = 20
decq = 1
sx=48
amty, amtx = 1, 1
capacity = 1e10
width = 0.1

# set up curves y = TKNQ, x = TKNB
PAIR = P("TKNB/TKNQ").sd(decb,decq)
price = 1 # TKNQ per TKNB
capacity = 1e6 # y = 1e6 TKNQ
oui = CarbonOrderUI.from_prices(PAIR, "TKNQ", price, price/(1+width), capacity, capacity)
curve = oui.yzABS(sx=sx, verbose=VERBOSE)
#print(curve)

# dy from dx
params  = (amty*10**decq, curve)             # amty = TKNQ
dx  = dx_from_dy(params, STB()) / 10**decb    # dx   = TKNB 
dx2 = oui.dxfromdy_f(amty)
print(f"dx={dx:0.6f} vs {dx2:0.6f} {oui.tknx} ==> diff={abs(dx/dx2-1)*100:0.4f}%")

# dx from dy
params  = (amtx*10**decb, curve)             # amtx = TKNB
dy = dy_from_dx(params, STB()) / 10**decq    # dy   = TKNQ 
dy2 = oui.dyfromdx_f(amtx)
print(f"dy={dy:0.6f} vs {dy2:0.6f} {oui.tkny} ==> diff={abs(dy/dy2-1)*100:0.4f}%")
dy_from_dx(params, STB())

dx=1.000027 vs 1.000000 TKNB ==> diff=0.0027%
dy=0.900000 vs 1.000000 TKNQ ==> diff=10.0000%


9

Note: the issues for `decb=0,1` are caused by the fact that the number of tokens expected is very small (1,10 respectively). The float amounts are $10^{-4}$ close to the expected value, but because they are below the int division rounds down, giving 1 token difference, which corresponds to 100% and 10% respectively.

In [18]:
28146000000*9999467920351890/281474989799959504785244160

0.9998935404038686

In [19]:
890090000000*316223491836353792/28147498980863303799508303872

9.99972921359663

### Loop over decimals

In [20]:
## SETUP Y = TKNQ, X = TKNB

# parameters
decb = 20
sx=40
amty, amtx = 1, 1
capacity = 1e12
width = 0.1

for decq in range(0,41):
    print(f"\ndecq = {decq}")

    # set up curves y = TKNQ, x = TKNB
    PAIR = P("TKNB/TKNQ").sd(decb,decq)
    price = 1 # TKNQ per TKNB
    capacity = 1e6 # y = 1e6 TKNQ
    oui = CarbonOrderUI.from_prices(PAIR, "TKNQ", price, price/(1+width), capacity, capacity)
    curve = oui.yzABS(sx=sx, verbose=VERBOSE)
    #print(curve)

    # dy from dx
    params  = (amty*10**decq, curve)             # amty = TKNQ
    dx  = dx_from_dy(params, STB()) / 10**decb    # dx   = TKNB 
    dx2 = oui.dxfromdy_f(amty)
    print(f"dx={dx:0.6f} vs {dx2:0.6f} {oui.tknx} ==> diff={abs(dx/dx2-1)*100:0.4f}%")

    # dx from dy
    params  = (amtx*10**decb, curve)             # amtx = TKNB
    dy = dy_from_dx(params, STB()) / 10**decq    # dy   = TKNQ 
    dy2 = oui.dyfromdx_f(amtx)
    print(f"dy={dy:0.6f} vs {dy2:0.6f} {oui.tkny} ==> diff={abs(dy/dy2-1)*100:0.4f}%")


decq = 0
dx=1.017529 vs 1.000000 TKNB ==> diff=1.7529%
dy=0.000000 vs 1.000000 TKNQ ==> diff=100.0000%

decq = 1
dx=1.004016 vs 1.000000 TKNB ==> diff=0.4016%
dy=0.900000 vs 1.000000 TKNQ ==> diff=10.0000%

decq = 2
dx=1.000931 vs 1.000000 TKNB ==> diff=0.0931%
dy=0.990000 vs 1.000000 TKNQ ==> diff=1.0000%

decq = 3
dx=1.000553 vs 1.000000 TKNB ==> diff=0.0553%
dy=0.999000 vs 1.000000 TKNQ ==> diff=0.1000%

decq = 4
dx=1.000203 vs 1.000000 TKNB ==> diff=0.0203%
dy=0.999700 vs 1.000000 TKNQ ==> diff=0.0300%

decq = 5
dx=1.000035 vs 1.000000 TKNB ==> diff=0.0035%
dy=0.999960 vs 1.000000 TKNQ ==> diff=0.0040%

decq = 6
dx=1.000021 vs 1.000000 TKNB ==> diff=0.0021%
dy=0.999978 vs 1.000000 TKNQ ==> diff=0.0022%

decq = 7
dx=1.000006 vs 1.000000 TKNB ==> diff=0.0006%
dy=0.999993 vs 1.000000 TKNQ ==> diff=0.0006%

decq = 8
dx=1.000001 vs 1.000000 TKNB ==> diff=0.0001%
dy=0.999999 vs 1.000000 TKNQ ==> diff=0.0001%

decq = 9
dx=1.000001 vs 1.000000 TKNB ==> diff=0.0001%
dy=0.999999 vs 1.000000

### Loop over prices (18/18)

In [21]:
## SETUP Y = TKNQ, X = TKNB

# parameters
decb = 18
decq = decb
sx=40
amtx = 1 # TKNB
capacity = 1e6
width = 0.1

for priceexp in range(-10,10):
    price = 10**priceexp
    amty = amtx*price # TKNQ
    print(f"\nprice = {price} TKNQ per TKNB [10**{priceexp}]")

    # set up curves y = TKNQ, x = TKNB
    PAIR = P("TKNB/TKNQ").sd(decb,decq)
    #price = 1 # TKNQ per TKNB
    capacity = 1e6 # y = 1e6 TKNQ
    oui = CarbonOrderUI.from_prices(PAIR, "TKNQ", price, price/(1+width), capacity*price, capacity*price)
    curve = oui.yzABS(sx=sx, verbose=VERBOSE)
    #print(curve)

    # dy from dx
    params  = (amty*10**decq, curve)             # amty = TKNQ
    dx  = dx_from_dy(params, STB()) / 10**decb    # dx   = TKNB 
    dx2 = oui.dxfromdy_f(amty)
    print(f"dx={dx:0.6f} vs {dx2:0.6f} {oui.tknx} ==> diff={abs(dx/dx2-1)*100:0.4f}%")

    # dx from dy
    params  = (amtx*10**decb, curve)             # amtx = TKNB
    dy = dy_from_dx(params, STB()) / 10**decq    # dy   = TKNQ 
    dy2 = oui.dyfromdx_f(amtx)
    print(f"dy={dy:0.6f} vs {dy2:0.6f} {oui.tkny} ==> diff={abs(dy/dy2-1)*100:0.4f}%")


price = 1e-10 TKNQ per TKNB [10**-10]
dx=1.000000 vs 1.000000 TKNB ==> diff=0.0000%
dy=0.000000 vs 0.000000 TKNQ ==> diff=0.0000%

price = 1e-09 TKNQ per TKNB [10**-9]
dx=1.000000 vs 1.000000 TKNB ==> diff=0.0000%
dy=0.000000 vs 0.000000 TKNQ ==> diff=0.0000%

price = 1e-08 TKNQ per TKNB [10**-8]
dx=1.000000 vs 1.000000 TKNB ==> diff=0.0000%
dy=0.000000 vs 0.000000 TKNQ ==> diff=0.0000%

price = 1e-07 TKNQ per TKNB [10**-7]
dx=1.000000 vs 1.000000 TKNB ==> diff=0.0000%
dy=0.000000 vs 0.000000 TKNQ ==> diff=0.0000%

price = 1e-06 TKNQ per TKNB [10**-6]
dx=1.000000 vs 1.000000 TKNB ==> diff=0.0000%
dy=0.000001 vs 0.000001 TKNQ ==> diff=0.0000%

price = 1e-05 TKNQ per TKNB [10**-5]
dx=1.000000 vs 1.000000 TKNB ==> diff=0.0000%
dy=0.000010 vs 0.000010 TKNQ ==> diff=0.0000%

price = 0.0001 TKNQ per TKNB [10**-4]
dx=1.000000 vs 1.000000 TKNB ==> diff=0.0000%
dy=0.000100 vs 0.000100 TKNQ ==> diff=0.0000%

price = 0.001 TKNQ per TKNB [10**-3]
dx=1.000000 vs 1.000000 TKNB ==> diff=0.0000%
dy=0

### Loop over prices (18/6)

In [22]:
## SETUP Y = TKNQ, X = TKNB

# parameters
decb = 18
decq = 6
sx=48
amtx = 1 # TKNB
capacity = 1e6
width = 0.1

for priceexp in range(-10,10):
    price = 10**priceexp
    amty = amtx*price # TKNQ
    print(f"\nprice = {price} TKNQ per TKNB [10**{priceexp}]")

    # set up curves y = TKNQ, x = TKNB
    PAIR = P("TKNB/TKNQ").sd(decb,decq)
    #price = 1 # TKNQ per TKNB
    capacity = 1e6 # y = 1e6 TKNQ
    oui = CarbonOrderUI.from_prices(PAIR, "TKNQ", price, price/(1+width), capacity*price, capacity*price)
    curve = oui.yzABS(sx=sx, verbose=VERBOSE)
    #print(curve)

    # dy from dx
    params  = (amty*10**decq, curve)             # amty = TKNQ
    dx  = dx_from_dy(params, STB()) / 10**decb    # dx   = TKNB 
    dx2 = oui.dxfromdy_f(amty)
    print(f"dx={dx:0.6f} vs {dx2:0.6f} {oui.tknx} ==> diff={abs(dx/dx2-1)*100:0.4f}%")

    # dx from dy
    params  = (amtx*10**decb, curve)             # amtx = TKNB
    dy = dy_from_dx(params, STB()) / 10**decq    # dy   = TKNQ 
    dy2 = oui.dyfromdx_f(amtx)
    print(f"dy={dy:0.6f} vs {dy2:0.6f} {oui.tkny} ==> diff={abs(dy/dy2-1)*100:0.4f}%")


price = 1e-10 TKNQ per TKNB [10**-10]
dx=1.001248 vs 1.000000 TKNB ==> diff=0.1248%
dy=0.000000 vs 0.000000 TKNQ ==> diff=100.0000%

price = 1e-09 TKNQ per TKNB [10**-9]
dx=1.000229 vs 1.000000 TKNB ==> diff=0.0229%
dy=0.000000 vs 0.000000 TKNQ ==> diff=100.0000%

price = 1e-08 TKNQ per TKNB [10**-8]
dx=1.000106 vs 1.000000 TKNB ==> diff=0.0106%
dy=0.000000 vs 0.000000 TKNQ ==> diff=100.0000%

price = 1e-07 TKNQ per TKNB [10**-7]
dx=1.000027 vs 1.000000 TKNB ==> diff=0.0027%
dy=0.000000 vs 0.000000 TKNQ ==> diff=100.0000%

price = 1e-06 TKNQ per TKNB [10**-6]
dx=1.000007 vs 1.000000 TKNB ==> diff=0.0007%
dy=0.000000 vs 0.000001 TKNQ ==> diff=100.0000%

price = 1e-05 TKNQ per TKNB [10**-5]
dx=1.000002 vs 1.000000 TKNB ==> diff=0.0002%
dy=0.000009 vs 0.000010 TKNQ ==> diff=10.0000%

price = 0.0001 TKNQ per TKNB [10**-4]
dx=1.000001 vs 1.000000 TKNB ==> diff=0.0001%
dy=0.000099 vs 0.000100 TKNQ ==> diff=1.0000%

price = 0.001 TKNQ per TKNB [10**-3]
dx=1.000000 vs 1.000000 TKNB ==> diff=0